In [ ]:
# import dependencies
import boto3

In [ ]:
# Get the fully qualified ARN from role name
def get_role_arn(role_name):
    try:
        iam = boto3.resource('iam')
        role = iam.Role(role_name)
        role.load()  # calls GetRole to load attributes
    except ClientError:
        print("Couldn't get role named %s."%role_name)
        raise
    else:
        return role.arn

In [ ]:
# recreates a policy
# if policy is attached to roles, it is firstly detached.
# if policy has additional versions ot the default, additional versions are deleted

def recreate_policy(policy_name, policy_rights):
    try:
        # format policy ARN based on policy name
        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()['Account']
        policy_arn = f'arn:aws:iam::{account_id}:policy/{policy_name}'
                
        iam_client = boto3.client('iam')
       
        # try to get policy
        policy_exists = True
        try:
            iam_client.get_policy(PolicyArn=policy_arn)
        except botocore.exceptions.ClientError as get_policy_ex:
                if get_policy_ex.response["Error"]["Code"] == "NoSuchEntity":
                    policy_exists = False
                else:
                    raise
                    
        # if policy exists, delete it            
        if policy_exists:
            # get policy versions and delete all non-default versions. 
            # Default version is deleted when policy itself is deleted.
            policy_versions_response = iam_client.list_policy_versions(PolicyArn=policy_arn)
            versions = policy_versions_response['Versions']
            for version in versions:
                version_id = version['VersionId']
                is_default = version['IsDefaultVersion']
                if not is_default:
                    iam_client.delete_policy_version(PolicyArn=policy_arn, VersionId=version_id)

            # detach all roles from policy (requirement for policy deletion)
            policy_entities = iam_client.list_entities_for_policy(PolicyArn=policy_arn, EntityFilter='Role')
            attached_roles = policy_entities['PolicyRoles']
            role_names = [role['RoleName'] for role in attached_roles]
            for role_name in role_names:
                iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
                
            #  delete policy. If it does not exist, handle error internally and continue
            iam_client.delete_policy(PolicyArn=policy_arn)
        
        # create policy
        create_policy_response = iam_client.create_policy(PolicyName=omics_iam_policy_name, 
                           Description="Policy for Amazon Omics demo", 
                           PolicyDocument=json.dumps(policy_rights))
        
        return create_policy_response
    
    except botocore.exceptions.ClientError:
        print("Couldn't re create policy named %s."%policy_name)
        raise

In [ ]:
def omics_cleanup():
    
# check dependencies and delete:

# 1.delete annotation stores
# 1.delete_annotation_store_response = omics_client.delete_annotation_store(name='tutorial_annotations_20230604t091130', force=True)

# 2.delete variant stores
# 2.delete_variant_store_response = omics_client.delete_variant_store(name='tutorial_variants_20230604t095034', force=True)
# 2.delete_variant_store_response = omics_client.delete_variant_store(name='tutorial_variants_20230604t091130', force=True)

# 3. delete sequence store read sets
# delete_sequence_store_readsets_response = omics_client.batch_delete_read_set(ids=['8707322872','8688949133','2422007294','923 9599169','2898902638','4174230264','4592435828','5246664496'], sequenceStoreId='4564575745')

# 4. delete sequence stores
# 4.delete_sequence_store_response = omics_client.delete_sequence_store(id='4564575745')

# 5. del_reference_response = omics_client.delete_reference(id='3647538932',referenceStoreId='1047795911')

# 6.delete reference stores
# 6.del_reference_store_response = omics_client.delete_reference_store(id='1047795911')